# Nubank Challenge
In this demo we will create a set of tables using dimensional modeling. We'll be using `sqlalchemy`, but we will not use it's interface.

We start from the original data we need to store:

In [1]:
import io

import pandas as pd
import sqlalchemy

In [2]:
engine = sqlalchemy.create_engine('mysql+mysqldb://root@localhost', pool_recycle=3600)
connection = engine.connect()

df_csv = """transaction_id;credit_card_id;transaction_date;merchant_name;total_value;installment_value;installments
1;11111111;2018-01-10T00:00:00;Colorful Soaps;19.99;19.99;1
2;22222222;2018-01-11T00:01:00;Cantina da Mamma;43.5;43.5;1
3;33333333;2018-01-12T01:02:00;Boulevard Hotel;129;129;1
4;11111111;2018-01-15T11:11:11;Micas Bar;225.9;75.3;3
5;11111111;2018-01-15T11:11:11;Micas Bar;225.9;75.3;3
6;11111111;2018-01-15T11:11:11;Micas Bar;225.9;75.3;3
7;22222222;2018-01-18T22:10:01;IPear Store;9999.99;9999.99;1
8;11111111;2018-02-20T21:08:32;Forrest Paintball;1337;1337;1
9;44444444;2018-02-22T00:05:30;Unicorn Costumes;100;50;2
10;44444444;2018-02-22T00:05:30;Unicorn Costumes;100;50;2"""

df_installments = pd.read_csv(io.StringIO(df_csv), sep=";")
df_customers = pd.DataFrame([
    {
        "first_name": "John", 
        "last_name": "Doe", 
        "birthday": pd.to_datetime('1999-01-01'),
        "city": "Zapala",
        "state": "Neuquén",
        "country": "Argentina",
        "number": 11111111,
        "card_brand": 'Visa',
        "expiration_date": pd.to_datetime('2019-01-01'),
    }, {
        "first_name": "Mary", 
        "last_name": "Jane", 
        "birthday": pd.to_datetime('1999-01-02'),
        "city": "Timbuktu",
        "state": "Tombouctou",
        "country": "Mali",
        "number": 22222222,
        "card_brand": 'Visa',
        "expiration_date": pd.to_datetime('2019-01-01'),
    }, {
        "first_name": "Joao", 
        "last_name": "Silva", 
        "birthday": pd.to_datetime('1999-01-03'),
        "city": "El Bolson",
        "state": "Rio Negro",
        "country": "Argentina",
        "number": 33333333,
        "card_brand": 'Visa',
        "expiration_date": pd.to_datetime('2019-01-01'),
    }, {
        "first_name": "Maria", 
        "last_name": "José", 
        "birthday": pd.to_datetime('1999-01-04'),
        "city": "Oslo",
        "state": "Østlandet",
        "country": "Norway",
        "number": 44444444,
        "card_brand": 'Visa',
        "expiration_date": pd.to_datetime('2019-01-01'),
    }
])

df_installments['transaction_date'] = pd.to_datetime(df_installments['transaction_date'])
df_installments = df_installments.merge(df_customers, left_on='credit_card_id', right_on="number")
df_transactions = df_installments.groupby(["credit_card_id", "transaction_date", "merchant_name", "total_value", "installment_value", "installments"]).first().reset_index()
df_installments

,transaction_id,credit_card_id,transaction_date,merchant_name,total_value,installment_value,installments,birthday,card_brand,city,country,expiration_date,first_name,last_name,number,state
0,1,11111111,2018-01-10 00:00:00,Colorful Soaps,19.99,19.99,1,1999-01-01,Visa,Zapala,Argentina,2019-01-01,John,Doe,11111111,Neuquén
1,4,11111111,2018-01-15 11:11:11,Micas Bar,225.90,75.30,3,1999-01-01,Visa,Zapala,Argentina,2019-01-01,John,Doe,11111111,Neuquén
2,5,11111111,2018-01-15 11:11:11,Micas Bar,225.90,75.30,3,1999-01-01,Visa,Zapala,Argentina,2019-01-01,John,Doe,11111111,Neuquén
3,6,11111111,2018-01-15 11:11:11,Micas Bar,225.90,75.30,3,1999-01-01,Visa,Zapala,Argentina,2019-01-01,John,Doe,11111111,Neuquén
4,8,11111111,2018-02-20 21:08:32,Forrest Paintball,1337.00,1337.00,1,1999-01-01,Visa,Zapala,Argentina,2019-01-01,John,Doe,11111111,Neuquén
5,2,22222222,2018-01-11 00:01:00,Cantina da Mamma,43.50,43.50,1,1999-01-02,Visa,Timbuktu,Mali,2019-01-01,Mary,Jane,22222222,Tombouctou
6,7,22222222,2018-01-18 22:10:01,IPear Store,9999.99,9999.99,1,1999-01-02,Visa,Timbuktu,Mali,2019-01-01,Mary,Jane,22222222,Tombouctou
7,3,33333333,2018-01-12 01:02:00,Boulevard Hotel,129.00,129.00,1,1999-01-03,Visa,El Bolson,Argentina,2019-01-01,Joao,Silva,33333333,Rio Negro
8,9,44444444,2018-02-22 00:05:30,Unicorn Costumes,100.00,50.00,2,1999-01-04,Visa,Oslo,Norway,2019-01-01,Maria,José,44444444,Østlandet
9,10,44444444,2018-02-22 00:05:30,Unicorn Costumes,100.00,50.00,2,1999-01-04,Visa,Oslo,Norway,2019-01-01,Maria,José,44444444,Østlandet


We will now create the tables based on the following statements:
```SQL
CREATE TABLE dates (
    date_id INT NOT NULL AUTO_INCREMENT,
    date DATE,
    PRIMARY KEY (date_id)
);

CREATE TABLE merchant (
    merchant_id INT NULL AUTO_INCREMENT,
    merchant_name VARCHAR(256),
    PRIMARY KEY (merchant_id)
);

CREATE TABLE customer (
    customer_id INT NOT NULL AUTO_INCREMENT,
    first_name VARCHAR(128),
    last_name VARCHAR(128),
    birthday DATE,
    city VARCHAR(128),
    state VARCHAR(64),
    country VARCHAR(64),
    PRIMARY KEY (customer_id)
);

CREATE TABLE credit_card (
    credit_card_id INT NOT NULL AUTO_INCREMENT,
    number INT,
    name VARCHAR(256),
    expiration_date DATE,
    card_brand INT,
    PRIMARY KEY (credit_card_id)
);

CREATE TABLE transactions (
    transaction_id INT NOT NULL AUTO_INCREMENT,
    customer_id INT,
    credit_card_id INT,
    merchant_id INT,
    transaction_date_id INT,
    installments_id INT,
    total_value DECIMAL(19, 4),
    installments INT,
    PRIMARY KEY (transaction_id),
    FOREIGN KEY (customer_id) REFERENCES customer (customer_id),
    FOREIGN KEY (credit_card_id) REFERENCES credit_card (credit_card_id),
    FOREIGN KEY (merchant_id) REFERENCES merchant (merchant_id),
    FOREIGN KEY (transaction_date_id) REFERENCES dates (date_id)
);

CREATE TABLE inistallments (
    transaction_id INT NOT NULL AUTO_INCREMENT,
    installment_number INT NOT NULL,
    customer_id INT,
    credit_card_id INT,
    merchant_id INT,
    transaction_date_id INT,
    transaction_id INT,
    installment_date_id INT,
    installments INT,
    installment_value DECIMAL(19, 4),
    total_value DECIMAL(19, 4),
    PRIMARY KEY (installments_id, installment_number),
    FOREIGN KEY (customer_id) REFERENCES customer (customer_id),
    FOREIGN KEY (credit_card_id) REFERENCES credit_card (credit_card_id),
    FOREIGN KEY (merchant_id) REFERENCES merchant (merchant_id),
    FOREIGN KEY (transaction_date_id) REFERENCES dates (date_id),
    FOREIGN KEY (installment_date_id) REFERENCES dates (date_id)
);
```


In [3]:
CREATE_DATABASE = """
    CREATE DATABASE unbank;
    USE unbank;
"""
result = connection.execute(CREATE_DATABASE)
result.close()


In [4]:
CREATE_TABLES = """
CREATE TABLE dates (
    date_id INT NOT NULL AUTO_INCREMENT,
    date DATE,
    PRIMARY KEY (date_id)
);

CREATE TABLE merchant (
    merchant_id INT NULL AUTO_INCREMENT,
    merchant_name VARCHAR(256),
    PRIMARY KEY (merchant_id)
);

CREATE TABLE customer (
    customer_id INT NOT NULL AUTO_INCREMENT,
    first_name VARCHAR(128),
    last_name VARCHAR(128),
    birthday DATE,
    city VARCHAR(128),
    state VARCHAR(64),
    country VARCHAR(64),
    PRIMARY KEY (customer_id)
);

CREATE TABLE credit_card (
    credit_card_id INT NOT NULL AUTO_INCREMENT,
    number INT,
    name VARCHAR(256),
    expiration_date DATE,
    card_brand VARCHAR(64),
    PRIMARY KEY (credit_card_id)
);

CREATE TABLE installments (
    transaction_id INT NOT NULL AUTO_INCREMENT,
    installment_number INT NOT NULL,
    customer_id INT,
    credit_card_id INT,
    merchant_id INT,
    transaction_date_id INT,
    installment_date_id INT,
    installments INT,
    installment_value DECIMAL(19, 4),
    total_value DECIMAL(19, 4),
    PRIMARY KEY (transaction_id, installment_number),
    FOREIGN KEY (customer_id) REFERENCES customer (customer_id),
    FOREIGN KEY (credit_card_id) REFERENCES credit_card (credit_card_id),
    FOREIGN KEY (merchant_id) REFERENCES merchant (merchant_id),
    FOREIGN KEY (transaction_date_id) REFERENCES dates (date_id),
    FOREIGN KEY (installment_date_id) REFERENCES dates (date_id)
);
"""
installments_columns = [
    "transaction_id", "installment_number", "customer_id", "credit_card_id", "merchant_id", "transaction_date_id", 
    "installment_date_id", "installments", "installment_value", "total_value"]
customer_columns = ["customer_id", "first_name", "last_name", "birthday", "city", "state", "country"]
merchant_columns = ["merchant_id", "merchant_name"]
credit_card_columns = ["credit_card_id", "number", "name", "expiration_date", "card_brand"]
dates_columns = ["date_id", "date"]
result = connection.execute(CREATE_TABLES)

Now we fill the dimension tables

In [5]:
def fill_dimensions(row, engine):
    connection = engine.connect()
    connection.execute("USE unbank;")
    
    # filling data table
    response = connection.execute("SELECT * FROM dates WHERE date = '{0}';".format(row["transaction_date"].strftime("%Y-%m-%d")))
    if not response.fetchall():
        response = connection.execute("INSERT INTO dates (date) VALUES ('{0}');".format(row["transaction_date"].strftime("%Y-%m-%d")))
        print("INERTED date: {0}".format(row["transaction_date"].strftime("%Y-%m-%d")))
        for i in range(1, row["installments"]):
            month_offset = (row["transaction_date"]+pd.DateOffset(months=i)).strftime("%Y-%m-%d")
            response = connection.execute("INSERT INTO dates (date) VALUES ('{0}');".format(month_offset))
            print("INERTED date: {0}".format(month_offset))
        
    
    # filling merchant table
    response = connection.execute("SELECT * FROM merchant WHERE merchant_name = '{0}';".format(row["merchant_name"]))
    if not response.fetchall():
        response = connection.execute("INSERT INTO merchant (merchant_name) VALUES ('{0}');".format(row["merchant_name"]))
        print("INERTED merchant: {0}".format(row["merchant_name"]))
    
    # filling customer table
    response = connection.execute("SELECT * FROM customer WHERE first_name = '{0}';".format(row["first_name"]))
    if not response.fetchall():
        response = connection.execute("""INSERT INTO customer 
                                      (first_name, last_name, birthday, city, state, country) VALUES 
                                      ('{0}', '{1}', '{2}', '{3}', '{4}', '{5}');"""\
                                      .format(row["first_name"], row["last_name"], row["birthday"].strftime("%Y-%m-%d"), row["city"], row["state"], row["country"]))
        print("INERTED customer: {0}".format(row["first_name"]))
    
    # filling credit_card table
    response = connection.execute("SELECT * FROM credit_card WHERE number = {0};".format(row["number"]))
    if not response.fetchall():
        response = connection.execute("""INSERT INTO credit_card
                                      (number, name, expiration_date, card_brand) VALUES 
                                      ({0}, '{1}', '{2}', '{3}');"""\
                                      .format(row["number"], row["first_name"], row["expiration_date"], row["card_brand"]))
        print("INERTED credit_card: {0}".format(row["number"]))
    
    connection.close()
    
    
df_installments.apply(fill_dimensions, axis=1, args=(engine,))
""

INERTED date: 2018-01-10
INERTED merchant: Colorful Soaps
INERTED customer: John
INERTED credit_card: 11111111
INERTED date: 2018-01-15
INERTED date: 2018-02-15
INERTED date: 2018-03-15
INERTED merchant: Micas Bar
INERTED date: 2018-02-20
INERTED merchant: Forrest Paintball
INERTED date: 2018-01-11
INERTED merchant: Cantina da Mamma
INERTED customer: Mary
INERTED credit_card: 22222222
INERTED date: 2018-01-18
INERTED merchant: IPear Store
INERTED date: 2018-01-12
INERTED merchant: Boulevard Hotel
INERTED customer: Joao
INERTED credit_card: 33333333
INERTED date: 2018-02-22
INERTED date: 2018-03-22
INERTED merchant: Unicorn Costumes
INERTED customer: Maria
INERTED credit_card: 44444444


''

Then we fill the fact tables

In [6]:
def fill_facts(row, engine):
    connection = engine.connect()
    connection.execute("USE unbank;")
    
    # collecting ids
    response = connection.execute("SELECT customer_id FROM customer WHERE first_name = '{0}';".format(row["first_name"]))
    customer_id = response.fetchall()[0][0]
    
    response = connection.execute("SELECT credit_card_id FROM credit_card WHERE number = {0};".format(row["number"]))
    credit_card_id = response.fetchall()[0][0]
    
    response = connection.execute("SELECT date_id FROM dates WHERE date = '{0}';".format(row["transaction_date"].strftime("%Y-%m-%d")))
    transaction_date_id = response.fetchall()[0][0]
    
    response = connection.execute("SELECT merchant_id FROM merchant WHERE merchant_name = '{0}';".format(row["merchant_name"]))
    merchant_id = response.fetchall()[0][0]
        
    # filling installments table
    installment_number = 1
    installment = [
            customer_id, credit_card_id, merchant_id, transaction_date_id, transaction_date_id, 
            row["total_value"], row["installments"], row["installment_value"], installment_number]
    response = connection.execute("""INSERT INTO installments
                          (customer_id, credit_card_id, merchant_id, transaction_date_id, installment_date_id, 
                          total_value, installments, installment_value, installment_number) 
                          VALUES ({0}, {1}, {2}, {3}, {4}, {5}, {6}, {7}, {8});"""\
                          .format(*installment))
    
    response = connection.execute("""SELECT transaction_id FROM installments WHERE
                            customer_id = {0} AND
                            credit_card_id = {1} AND
                            merchant_id = {2} AND
                            transaction_date_id = {3} AND
                            installment_date_id = {4} AND
                            total_value = {5} AND
                            installments = {6} AND
                            installment_value = {7} AND
                            installment_number = {8};"""\
                            .format(*installment))
    transaction_id = response.fetchall()[0][0]
    
    for installment_number in range(2, row["installments"]+1):
        month_offset = (row["transaction_date"]+pd.DateOffset(months=installment_number-1)).strftime("%Y-%m-%d")
        response = connection.execute("""SELECT date_id FROM dates WHERE date = '{0}'""".format(month_offset))
        installment_date_id = response.fetchall()[0][0]
        installment = [
            customer_id, credit_card_id, merchant_id, transaction_id, transaction_date_id,
            installment_date_id, row["total_value"], row["installments"], row["installment_value"], 
            installment_number]
        response = connection.execute("""INSERT INTO installments
                                      (customer_id, credit_card_id, merchant_id, transaction_id, 
                                      transaction_date_id, installment_date_id, total_value, 
                                      installments, installment_value, installment_number) VALUES 
                                      ({0}, {1}, {2}, {3}, {4}, {5}, {6}, {7}, {8}, {9});"""\
                                      .format(*installment))
    connection.close()
    
df_transactions.apply(fill_facts, axis=1, args=(engine,))
""

''

In [7]:
pd.DataFrame(connection.execute("SELECT * FROM installments").fetchall(), columns=installments_columns)

,transaction_id,installment_number,customer_id,credit_card_id,merchant_id,transaction_date_id,installment_date_id,installments,installment_value,total_value
0,1,1,1,1,1,1,1,1,19.9900,19.9900
1,2,1,1,1,2,2,2,3,75.3000,225.9000
2,2,2,1,1,2,2,3,3,75.3000,225.9000
3,2,3,1,1,2,2,4,3,75.3000,225.9000
4,3,1,1,1,3,5,5,1,1337.0000,1337.0000
5,4,1,2,2,4,6,6,1,43.5000,43.5000
6,5,1,2,2,5,7,7,1,9999.9900,9999.9900
7,6,1,3,3,6,8,8,1,129.0000,129.0000
8,7,1,4,4,7,9,9,2,50.0000,100.0000
9,7,2,4,4,7,9,10,2,50.0000,100.0000


We can query the installments table joining it with the dimensions.

In [22]:
installments_join = """
    SELECT * FROM installments 
    JOIN customer ON installments.customer_id = customer.customer_id
    JOIN merchant ON installments.merchant_id = merchant.merchant_id
    JOIN credit_card ON installments.credit_card_id = credit_card.credit_card_id
    JOIN dates AS d1 ON installments.transaction_date_id = d1.date_id
    JOIN dates AS d2 ON installments.installment_date_id = d2.date_id
"""

all_installment_columns = installments_columns+customer_columns+merchant_columns+credit_card_columns+["date_id", "transaction_date"]+["date_id", "installment_date"]
df = pd.DataFrame(connection.execute(installments_join).fetchall(), columns=all_installment_columns).drop(["customer_id", "merchant_id", "credit_card_id", "transaction_date_id", "installment_date_id", "date_id"], axis=1)
df["installment_date"] = pd.to_datetime(df["installment_date"])
df["transaction_date"] = pd.to_datetime(df["transaction_date"])
df

,transaction_id,installment_number,installments,installment_value,total_value,first_name,last_name,birthday,city,state,country,merchant_name,number,name,expiration_date,card_brand,transaction_date,installment_date
0,1,1,1,19.9900,19.9900,John,Doe,1999-01-01,Zapala,Neuquén,Argentina,Colorful Soaps,11111111,John,2019-01-01,Visa,2018-01-10,2018-01-10
1,2,1,3,75.3000,225.9000,John,Doe,1999-01-01,Zapala,Neuquén,Argentina,Micas Bar,11111111,John,2019-01-01,Visa,2018-01-15,2018-01-15
2,2,2,3,75.3000,225.9000,John,Doe,1999-01-01,Zapala,Neuquén,Argentina,Micas Bar,11111111,John,2019-01-01,Visa,2018-01-15,2018-02-15
3,2,3,3,75.3000,225.9000,John,Doe,1999-01-01,Zapala,Neuquén,Argentina,Micas Bar,11111111,John,2019-01-01,Visa,2018-01-15,2018-03-15
4,3,1,1,1337.0000,1337.0000,John,Doe,1999-01-01,Zapala,Neuquén,Argentina,Forrest Paintball,11111111,John,2019-01-01,Visa,2018-02-20,2018-02-20
5,4,1,1,43.5000,43.5000,Mary,Jane,1999-01-02,Timbuktu,Tombouctou,Mali,Cantina da Mamma,22222222,Mary,2019-01-01,Visa,2018-01-11,2018-01-11
6,5,1,1,9999.9900,9999.9900,Mary,Jane,1999-01-02,Timbuktu,Tombouctou,Mali,IPear Store,22222222,Mary,2019-01-01,Visa,2018-01-18,2018-01-18
7,6,1,1,129.0000,129.0000,Joao,Silva,1999-01-03,El Bolson,Rio Negro,Argentina,Boulevard Hotel,33333333,Joao,2019-01-01,Visa,2018-01-12,2018-01-12
8,7,1,2,50.0000,100.0000,Maria,José,1999-01-04,Oslo,Østlandet,Norway,Unicorn Costumes,44444444,Maria,2019-01-01,Visa,2018-02-22,2018-02-22
9,7,2,2,50.0000,100.0000,Maria,José,1999-01-04,Oslo,Østlandet,Norway,Unicorn Costumes,44444444,Maria,2019-01-01,Visa,2018-02-22,2018-03-22


In [23]:
df[df["first_name"] == "John"]

,transaction_id,installment_number,installments,installment_value,total_value,first_name,last_name,birthday,city,state,country,merchant_name,number,name,expiration_date,card_brand,transaction_date,installment_date
0,1,1,1,19.9900,19.9900,John,Doe,1999-01-01,Zapala,Neuquén,Argentina,Colorful Soaps,11111111,John,2019-01-01,Visa,2018-01-10,2018-01-10
1,2,1,3,75.3000,225.9000,John,Doe,1999-01-01,Zapala,Neuquén,Argentina,Micas Bar,11111111,John,2019-01-01,Visa,2018-01-15,2018-01-15
2,2,2,3,75.3000,225.9000,John,Doe,1999-01-01,Zapala,Neuquén,Argentina,Micas Bar,11111111,John,2019-01-01,Visa,2018-01-15,2018-02-15
3,2,3,3,75.3000,225.9000,John,Doe,1999-01-01,Zapala,Neuquén,Argentina,Micas Bar,11111111,John,2019-01-01,Visa,2018-01-15,2018-03-15
4,3,1,1,1337.0000,1337.0000,John,Doe,1999-01-01,Zapala,Neuquén,Argentina,Forrest Paintball,11111111,John,2019-01-01,Visa,2018-02-20,2018-02-20


In [28]:
# df[df["first_name"] == "John"].groupby("installment_date", pd.Grouper(freq="M"))["installment_value"].sum()
df[df["first_name"] == "John"][["installment_date", "installment_value"]].set_index("installment_date").resample("M").sum()

,installment_value
installment_date,
2018-01-31,95.2900
2018-02-28,1412.3000
2018-03-31,75.3000


In [47]:
installments_join = """
    SELECT EXTRACT(YEAR FROM d2.date), EXTRACT(MONTH FROM d2.date), SUM(installment_value) FROM installments 
    JOIN credit_card ON installments.credit_card_id = credit_card.credit_card_id
    JOIN dates AS d2 ON installments.installment_date_id = d2.date_id
    WHERE credit_card.number = 11111111
    GROUP BY EXTRACT(YEAR FROM d2.date), EXTRACT(MONTH FROM d2.date);
"""
df = pd.DataFrame(connection.execute(installments_join).fetchall(), columns=["installment_year", "installment_month", "installment_value"])
df

,installment_year,installment_month,installment_value
0,2018,1,95.2900
1,2018,2,1412.3000
2,2018,3,75.3000


,transaction_id,installment_number,installments,installment_value,total_value,first_name,last_name,birthday,city,state,country,merchant_name,number,name,expiration_date,card_brand,transaction_date,installment_date
